In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ames-housing-dataset/AmesHousing.csv


                                                ******PREDICTING HOUSING PRICE IN AMES , LOWA*******

In [2]:
house_data = pd.read_csv('/kaggle/input/ames-housing-dataset/AmesHousing.csv')
print(house_data.head(3))

   Order        PID  MS SubClass MS Zoning  Lot Frontage  Lot Area Street  \
0      1  526301100           20        RL         141.0     31770   Pave   
1      2  526350040           20        RH          80.0     11622   Pave   
2      3  526351010           20        RL          81.0     14267   Pave   

  Alley Lot Shape Land Contour  ... Pool Area Pool QC  Fence Misc Feature  \
0   NaN       IR1          Lvl  ...         0     NaN    NaN          NaN   
1   NaN       Reg          Lvl  ...         0     NaN  MnPrv          NaN   
2   NaN       IR1          Lvl  ...         0     NaN    NaN         Gar2   

  Misc Val Mo Sold Yr Sold Sale Type  Sale Condition  SalePrice  
0        0       5    2010       WD           Normal     215000  
1        0       6    2010       WD           Normal     105000  
2    12500       6    2010       WD           Normal     172000  

[3 rows x 82 columns]


In [3]:
X = house_data.columns
print(X)

Index(['Order', 'PID', 'MS SubClass', 'MS Zoning', 'Lot Frontage', 'Lot Area',
       'Street', 'Alley', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1',
       'Condition 2', 'Bldg Type', 'House Style', 'Overall Qual',
       'Overall Cond', 'Year Built', 'Year Remod/Add', 'Roof Style',
       'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Mas Vnr Type',
       'Mas Vnr Area', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual',
       'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1',
       'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF',
       'Heating', 'Heating QC', 'Central Air', 'Electrical', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath',
       'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr',
       'Kitchen AbvGr', 'Kitchen Qual', 'TotRms AbvGrd', 'Functional',
       'Fireplaces', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt',
      

                                                        ****FINDING MISSING VALUES****

In [4]:
missing_cols = [col for col in house_data if house_data[col].isnull().any()]
print(missing_cols) 

['Lot Frontage', 'Alley', 'Mas Vnr Type', 'Mas Vnr Area', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin SF 1', 'BsmtFin Type 2', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', 'Electrical', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Fireplace Qu', 'Garage Type', 'Garage Yr Blt', 'Garage Finish', 'Garage Cars', 'Garage Area', 'Garage Qual', 'Garage Cond', 'Pool QC', 'Fence', 'Misc Feature']


In [5]:
for col in house_data:
    if house_data[col].isnull().any():
        print(house_data[col].isnull().sum())

490
2732
1775
23
80
80
83
80
1
81
1
1
1
1
2
2
1422
157
159
159
1
1
159
159
2917
2358
2824


In [6]:
modifi_house_data = house_data.dropna(thresh=house_data.shape[0] - 400, axis=1)
modified_house_data = modifi_house_data.drop('SalePrice',axis=1)
target = modifi_house_data['SalePrice']
print(modified_house_data)

      Order        PID  MS SubClass MS Zoning  Lot Area Street Lot Shape  \
0         1  526301100           20        RL     31770   Pave       IR1   
1         2  526350040           20        RH     11622   Pave       Reg   
2         3  526351010           20        RL     14267   Pave       IR1   
3         4  526353030           20        RL     11160   Pave       Reg   
4         5  527105010           60        RL     13830   Pave       IR1   
...     ...        ...          ...       ...       ...    ...       ...   
2925   2926  923275080           80        RL      7937   Pave       IR1   
2926   2927  923276100           20        RL      8885   Pave       IR1   
2927   2928  923400125           85        RL     10441   Pave       Reg   
2928   2929  924100070           20        RL     10010   Pave       Reg   
2929   2930  924151050           60        RL      9627   Pave       Reg   

     Land Contour Utilities Lot Config  ... Open Porch SF Enclosed Porch  \
0          

In [7]:
object_cols = [col for col in modified_house_data if modified_house_data[col].dtype == 'object']
print(object_cols)

['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities', 'Lot Config', 'Land Slope', 'Neighborhood', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Exterior 1st', 'Exterior 2nd', 'Exter Qual', 'Exter Cond', 'Foundation', 'Bsmt Qual', 'Bsmt Cond', 'Bsmt Exposure', 'BsmtFin Type 1', 'BsmtFin Type 2', 'Heating', 'Heating QC', 'Central Air', 'Electrical', 'Kitchen Qual', 'Functional', 'Garage Type', 'Garage Finish', 'Garage Qual', 'Garage Cond', 'Paved Drive', 'Sale Type', 'Sale Condition']


In [8]:
numerical_cols = [col for col in modified_house_data if modified_house_data[col].dtype in ['int64','float64']]
print(numerical_cols)

['Order', 'PID', 'MS SubClass', 'Lot Area', 'Overall Qual', 'Overall Cond', 'Year Built', 'Year Remod/Add', 'Mas Vnr Area', 'BsmtFin SF 1', 'BsmtFin SF 2', 'Bsmt Unf SF', 'Total Bsmt SF', '1st Flr SF', '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Bsmt Full Bath', 'Bsmt Half Bath', 'Full Bath', 'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd', 'Fireplaces', 'Garage Yr Blt', 'Garage Cars', 'Garage Area', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch', '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'Mo Sold', 'Yr Sold']


In [9]:
from sklearn.impute import SimpleImputer

num_imputer = SimpleImputer(strategy='mean')
obj_imputer = SimpleImputer(strategy='most_frequent')

imputed_num_cols = pd.DataFrame(num_imputer.fit_transform(modified_house_data[numerical_cols]),columns=numerical_cols)
imputed_obj_cols = pd.DataFrame(obj_imputer.fit_transform(modified_house_data[object_cols]),columns=object_cols)

imputed_house_data = pd.concat([modified_house_data.drop(numerical_cols + object_cols, axis=1),imputed_num_cols,imputed_obj_cols], axis=1)
print(imputed_house_data)


       Order          PID  MS SubClass  Lot Area  Overall Qual  Overall Cond  \
0        1.0  526301100.0         20.0   31770.0           6.0           5.0   
1        2.0  526350040.0         20.0   11622.0           5.0           6.0   
2        3.0  526351010.0         20.0   14267.0           6.0           6.0   
3        4.0  526353030.0         20.0   11160.0           7.0           5.0   
4        5.0  527105010.0         60.0   13830.0           5.0           5.0   
...      ...          ...          ...       ...           ...           ...   
2925  2926.0  923275080.0         80.0    7937.0           6.0           6.0   
2926  2927.0  923276100.0         20.0    8885.0           5.0           5.0   
2927  2928.0  923400125.0         85.0   10441.0           5.0           5.0   
2928  2929.0  924100070.0         20.0   10010.0           5.0           5.0   
2929  2930.0  924151050.0         60.0    9627.0           7.0           5.0   

      Year Built  Year Remod/Add  Mas V

In [10]:
low_object_cols = [col for col in imputed_house_data[object_cols] if imputed_house_data[col].nunique()<50]
high_object_cols = [col for col in imputed_house_data[object_cols] if imputed_house_data[col].nunique()>=50]

In [11]:
from sklearn.preprocessing import OneHotEncoder , OrdinalEncoder

simple_encoder = OrdinalEncoder()
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)

se_house_data = modified_house_data.copy()
se_house_data[high_object_cols] = simple_encoder.fit_transform(modified_house_data[high_object_cols])

modified_house_data = se_house_data.copy()

oh_encoded_data = pd.DataFrame(encoder.fit_transform(modified_house_data[high_object_cols]))
oh_encoded_data.index = modified_house_data.index

num_house_data = modified_house_data.drop(object_cols,axis=1)
final_house_data = pd.concat([num_house_data,oh_encoded_data],axis=1)


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
for col in final_house_data:
    if final_house_data[col].dtype == 'object':
        print(final_house_data[col])

In [ ]:
from sklearn.decomposition import PCA

X = final_house_data.copy()
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

pca = PCA(n_components=0.95)  # Retains 95% of the variance
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [17]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split

xgb = XGBRegressor(n_estimators=500,early_stopping_rounds=5,learning_rate=0.01)
x_train,x_test,y_train,y_test = train_test_split(final_house_data,target,test_size=0.2,random_state=0)

model = xgb.fit(x_train,y_train,eval_set=[(x_test, y_test)], verbose=False)
score = model.score(x_test,y_test)
print("xgb : ",score)

xgb :  0.8986884671225879
